## 数据探索

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('broadband.csv')
df.head()

,CUST_ID,GENDER,AGE,TENURE,CHANNEL,AUTOPAY,ARPB_3M,CALL_PARTY_CNT,DAY_MOU,AFTERNOON_MOU,NIGHT_MOU,AVG_CALL_LENGTH,BROADBAND
0,63,1,34,27,2,0,203,0,0.0,0.0,0.0,3.04,1
1,64,0,62,58,1,0,360,0,0.0,1910.0,0.0,3.30,1
2,65,1,39,55,3,0,304,0,437.2,200.3,0.0,4.92,0
3,66,1,39,55,3,0,304,0,437.2,182.8,0.0,4.92,0
4,67,1,39,55,3,0,304,0,437.2,214.5,0.0,4.92,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114 entries, 0 to 1113
Data columns (total 13 columns):
CUST_ID            1114 non-null int64
GENDER             1114 non-null int64
AGE                1114 non-null int64
TENURE             1114 non-null int64
CHANNEL            1114 non-null int64
AUTOPAY            1114 non-null int64
ARPB_3M            1114 non-null int64
CALL_PARTY_CNT     1114 non-null int64
DAY_MOU            1114 non-null float64
AFTERNOON_MOU      1114 non-null float64
NIGHT_MOU          1114 non-null float64
AVG_CALL_LENGTH    1114 non-null float64
BROADBAND          1114 non-null int64
dtypes: float64(4), int64(9)
memory usage: 113.2 KB


###### 参数说明
本代码文件只为演示随机森林的用法和调优方法，所以数据参数我们只需关注最后一个 

broadband 即可：0-离开，1-留存；

其他自变量意思可不做探究，毕竟真实工作中的数据集也完全不一样。

In [4]:
# 列名全部换成小写
df.rename(str.lower, axis='columns', inplace=True)
df.sample()

,cust_id,gender,age,tenure,channel,autopay,arpb_3m,call_party_cnt,day_mou,afternoon_mou,night_mou,avg_call_length,broadband
1036,669,0,64,3,1,0,149,0,0.0,0.0,0.0,2.2,0


In [5]:
# 查看因变量 broadband 分布情况，看是否存在不平衡
from collections import Counter
print('Broadband: ', Counter(df['broadband'])) 
## Broadband:  Counter({0: 908, 1: 206}) 比较不平衡。
## 根据原理部分，可知随机森林是处理数据不平衡问题的利器

Broadband:  Counter({0: 908, 1: 206})


## 拆分测试集与训练集

In [6]:
y = df['broadband'] 
X = df.iloc[:, 1:-1] 
# 客户 id 没有用，故丢弃 cust_id, 
## 0 可以表示第一列和最后一列，所以 1:-1 自动丢弃了第一列的客户 id 与最后一列的因变量

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=0.4, random_state=12345)

## 决策树建模
为了对比随机森林效果，也可作为基础温习

In [7]:
import sklearn.tree as tree

# 直接使用交叉网格搜索来优化决策树模型，边训练边优化
from sklearn.model_selection import GridSearchCV
# 网格搜索的参数：正常决策树建模中的参数 - 评估指标，树的深度，最小拆分的叶子样本数与树的深度
param_grid = {'criterion': ['entropy', 'gini'],
             'max_depth': [2, 3, 4, 5, 6, 7, 8],
             'min_samples_split': [4, 8, 12, 16, 20, 24, 28]} 
                # 通常来说，十几层的树已经是比较深了

In [8]:
clf = tree.DecisionTreeClassifier()  # 定义一棵树
clfcv = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc',
                    cv=4) # 传入模型，网格搜索的参数，评估指标，cv交叉验证的次数
                          ## 这里也只是定义，还没有开始训练模型

In [9]:
clfcv.fit(X=X_train, y=y_train)

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'min_samples_split': [4, 8, 12, 16, 20, 24, 28]},
             scoring='roc_auc')

In [10]:
# 使用模型来对测试集进行预测
test_est = clfcv.predict(X_test)

# 模型评估
import sklearn.metrics as metrics

print("决策树准确度:")
print(metrics.classification_report(y_test,test_est)) # 该矩阵表格其实作用不大
print("决策树 AUC:")
fpr_test, tpr_test, th_test = metrics.roc_curve(y_test, test_est)
print('AUC = %.4f' %metrics.auc(fpr_test, tpr_test))

# AUC 大于 0.5 是最基本的要求，可见模型精度还是比较糟糕的
# 决策树的调优技巧不多展开，将在随机森林调优部分展示

决策树准确度:
              precision    recall  f1-score   support

           0       0.84      0.99      0.91       359
           1       0.79      0.22      0.34        87

    accuracy                           0.84       446
   macro avg       0.82      0.60      0.62       446
weighted avg       0.83      0.84      0.80       446

决策树 AUC:
AUC = 0.6022


## 随机森林建模

In [11]:
# 一样是直接使用网格搜索
param_grid = {
    'criterion':['entropy','gini'],
    'max_depth':[5, 6, 7, 8],    # 深度：这里是森林中每棵决策树的深度
    'n_estimators':[11,13,15],  # 决策树个数-随机森林特有参数
    'max_features':[0.3,0.4,0.5], # 每棵决策树使用的变量占比-随机森林特有参数（结合原理）
    'min_samples_split':[4,8,12,16]  # 叶子的最小拆分样本量
}

In [12]:
import sklearn.ensemble as ensemble # ensemble learning: 集成学习

In [13]:
rfc = ensemble.RandomForestClassifier()
rfc_cv = GridSearchCV(estimator=rfc, param_grid=param_grid,
                      scoring='roc_auc', cv=4)
rfc_cv.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [5, 6, 7, 8],
                         'max_features': [0.3, 0.4, 0.5],
                         'min_samples_split': [4, 8, 12, 16],
                         'n_estimators': [11, 13, 15]},
             scoring='roc_auc')

In [14]:
# 使用随机森林对测试集进行预测
test_est = rfc_cv.predict(X_test)
print('随机森林精确度...')
print(metrics.classification_report(test_est, y_test))
print('随机森林 AUC...')
fpr_test, tpr_test, th_test = metrics.roc_curve(test_est, y_test) # 构造 roc 曲线
print('AUC = %.4f' %metrics.auc(fpr_test, tpr_test))
# AUC ，即预测类模型的精度大大提升

随机森林精确度...
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       405
           1       0.38      0.80      0.52        41

    accuracy                           0.86       446
   macro avg       0.68      0.84      0.72       446
weighted avg       0.92      0.86      0.88       446

随机森林 AUC...
AUC = 0.8358


In [15]:
# 查看最佳参数，看是否在决策边界上，还需重新设置网格搜索参数
rfc_cv.best_params_

# """
# 原来的参数
# param_grid = {
#     'criterion':['entropy','gini'],
#     'max_depth':[5, 6, 7, 8],    # 深度：这里是森林中每棵决策树的深度
#     'n_estimators':[11,13,15],  # 决策树个数-随机森林特有参数
#     'max_features':[0.3,0.4,0.5], # 每棵决策树使用的变量占比-随机森林特有参数（结合原理）
#     'min_samples_split':[4,8,12,16]  # 叶子的最小拆分样本量
# }

# 打印出来的最佳参数
# {'criterion': 'gini',
#  'max_depth': 8,  在最大值边界上，所以这个参数的最大值范围应该再调大
#  'max_features': 0.5,  也在最大值边界上，说明这个参数的最小值范围应该再调大
#  'min_samples_split': 4, 同理，在最小边界上，可考虑把范围调小
#  'n_estimators': 15 同理，在最大边界上，可以适当调大范围}
# """

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 0.4,
 'min_samples_split': 4,
 'n_estimators': 15}

In [16]:
# 调整决策边界，这里只是做示范，实际业务中有时候调整一个上午也是正常的。
param_grid = {
    'criterion':['entropy','gini'],
    'max_depth':[7, 8, 10, 12], # 前面的 5，6 也可以适当的去掉，反正已经没有用了
    'n_estimators':[11, 13, 15, 17, 19],  #决策树个数-随机森林特有参数
    'max_features':[0.4, 0.5, 0.6, 0.7], #每棵决策树使用的变量占比-随机森林特有参数
    'min_samples_split':[2, 3, 4, 8, 12, 16]  # 叶子的最小拆分样本量
}

# 重复上述步骤，可写成函数供快捷调用
rfc_cv = GridSearchCV(estimator=rfc, param_grid=param_grid,
                      scoring='roc_auc', cv=4)
rfc_cv.fit(X_train, y_train)
# 使用随机森林对测试集进行预测
test_est = rfc_cv.predict(X_test)
print('随机森林精确度...')
print(metrics.classification_report(test_est, y_test))
print('随机森林 AUC...')
fpr_test, tpr_test, th_test = metrics.roc_curve(test_est, y_test) # 构造 roc 曲线
print('AUC = %.4f' %metrics.auc(fpr_test, tpr_test))
# 这里的 auc 只提升了一点点，在实际情况中有时候能得到如 0.5 这样很不错的提升

随机森林精确度...
              precision    recall  f1-score   support

           0       0.97      0.88      0.92       397
           1       0.45      0.80      0.57        49

    accuracy                           0.87       446
   macro avg       0.71      0.84      0.75       446
weighted avg       0.91      0.87      0.88       446

随机森林 AUC...
AUC = 0.8375


In [17]:
rfc_cv.best_params_ 

{'criterion': 'gini',
 'max_depth': 12,
 'max_features': 0.6,
 'min_samples_split': 2,
 'n_estimators': 19}